In [1]:
# libraries
import json
import lzma
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from IPython.core.display import display, HTML
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords  
import nltk
nltk.download('stopwords')
import matplotlib.pyplot as plt
import seaborn as sns
import random
sns.set()

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# pd.options.display.max_columns = 999
# pd.options.display.max_rows = 999

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/qilongxin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df_nc = pd.read_csv('df_head_and_body_only.csv') #case data without any preprocessing
df_nc.head()

,Unnamed: 0,headnote,opinion_text
0,0,"SUPREME COURT,\nRALEIGH, (N. C.)\nJULY, 1810.\...","A majority of the court, therefore, being of t..."
1,1,"Case No. 4,170.\nDUNLOP et al. v. WEST.\n[Brun...",PER CURIAM.\nIf the sheriff or marshal seizes ...
2,2,"Case No. 4,954.\nThe FORTUNA.\n[1 Brock. 299.]...","MARSHALL. Circuit Justice.\nThe Fortuna, a ves..."
3,3,D. K. FUTCH v. ATLANTIC COAST LINE RAILROAD CO...,"Walker, J.\nThe grievance alleged by the plain..."
4,4,"MRS. SUSANNA WILLIAMS v. C. G. BAILEY, B. R. B...","Hoke, J.\nOn the hearing it appeared that C. G..."


In [5]:
# number of baseline cases to inspect 
baseline_size = 100

# randomly select 100 cases
baseline_data = df_nc.sample(n=baseline_size,replace=False, random_state=1)

#get text and summary 
baseline_text,baseline_summary = baseline_data.opinion_text.tolist(),baseline_data.headnote.tolist()


In [6]:
from summarizer import Summarizer
#https://github.com/dmmiller612/bert-extractive-summarizer
import rouge 

model = Summarizer()
rouge = rouge.Rouge()

# a function to get all scores from true summaries and generated summaries 
def getRougeScore(y_true,y_pred,metric='rouge-2',score='f'): 
    scores = np.zeros(len(y_true))
    for i in range(len(y_pred)):
        if y_pred[i] == '': # account for the possiblity that BERT fails and returns empty strings
            scores[i] = 0
            next
        else: 
            sc = rouge.get_scores(y_true[i], y_pred[i])
            scores[i] = sc[0][metric][score]
    print(np.mean(scores))
    return scores 

# a class to initialize, apply, and report baseline pre-trained BERT model performance
class Baseline_BERT_model(): 
    def __init__ (self,min_length=60): 
        super(Baseline_BERT_model,self).__init__()
        self.min_length = min_length
        
    def evaluate(self,texts,summaries):  
        results = []
        for i,text in enumerate(texts): 
            results.append(model(text,min_length=self.min_length))
        self.results = results
        self.summaries = summaries 
        return results 
    
    def reportEvaluation(self,metric='rouge-2',score='f'): 
        sc = getRougeScore(self.summaries,self.results,metric,score) 
        return sc
    def inspectOneSummary(self):
        # randomly inspect one summary 
        print(self.results[random.randint(0,len(self.results)-1)])
        

In [40]:
bert = Baseline_BERT_model()

_ = bert.evaluate(baseline_text,baseline_summary)
print('The average Rouge 1 F-scores on 100 random cases is')
_ = bert.reportEvaluation(metric='rouge-1')
print('The average Rouge 2 F-scores on 100 random cases is')
_ = bert.reportEvaluation()

The average Rouge 1 F-scores on 100 random cases is
0.2147327799704312
The average Rouge 2 F-scores on 100 random cases is
0.07484249324761535


In [9]:
bert.inspectOneSummary()

It appears therefrom that the plaintiff admits the purchase of the sewing machines at the price stated, but seeks to recover damages because, as it alleges, the defendant agreed to furnish the plaintiff an experienced and successful agent to sell the machines, and that said agent embezzled the horse and buggy furnished him and ran away with certain collections made by him. When furnished, the agent became the agent of the plaintiff, and any loss from his misconduct falls upon the plaintiff. He could not be the agent of both the plaintiff and defendant in selling machines for the plaintiff. The plaintiff company was engaged in the business of buying and retailing them. It is true' the reply denies that the plaintiff owes the sum set up in the counterclaim, but that is a conclusion of law, for the purchase and price of the sewing machines are admitted, and the liability therefor is denied solely on the ground that the agent recommended by the defendant and employed by plaintiff proved to